In [1]:
import math

deg2rad = lambda x : math.pi * x / 180.

LONG = [
  [
    [200, 500, 200, deg2rad(180), deg2rad(-5)],
    [500, 350, 100, deg2rad(0), deg2rad(-5)],
    [467.70, 449.56],
    "Long 1"
  ],[
    [100, -400, 100, deg2rad(30), deg2rad(0)],
    [500, -700, 0, deg2rad(150), deg2rad(0)],
    [649.52, 636.12],
    "Long 2"
  ],[
    [-200, 200, 250, deg2rad(240), deg2rad(15)],
    [ 500, 800,   0, deg2rad(45), deg2rad(15)],
    [1088.10, 1063.41],
    "Long 3"
  ],[
    [-300, 1200, 350, deg2rad(160), deg2rad(0)],
    [1000,  200,   0, deg2rad(30), deg2rad(0)],
    [1802.60, 1789.21],
    "Long 4"
  ],[
    [-500, -300, 600, deg2rad(150), deg2rad(10)],
    [1200,  900, 100, deg2rad(300), deg2rad(10)],
    [2245.14, 2216.40],
    "Long 5"
  ]
]

SHORT = [
  [
    [120, -30, 250, deg2rad(100), deg2rad(-10)],
    [220, 150, 100, deg2rad(300), deg2rad(-10)],
    [588.60, 583.47],
    "Short 1"
  ],[
    [380, 230, 200, deg2rad(30), deg2rad(0)],
    [280, 150,  30, deg2rad(200), deg2rad(0)],
    [667.71, 658.53],
    "Short 2"
  ],[
    [-80, 10, 250, deg2rad(20), deg2rad(0)],
    [ 50, 70,   0, deg2rad(240), deg2rad(0)],
    [979.34, 968.25],
    "Short 3"
  ],[
    [400, -250, 600, deg2rad(350), deg2rad(0)],
    [600, -150, 300, deg2rad(150), deg2rad(0)],
    [1169.73, 1161.55],
    "Short 4"
  ],[
    [-200, -200, 450, deg2rad(340), deg2rad(0)],
    [-300,  -80, 100, deg2rad(100), deg2rad(0)],
    [1367.56, 1354.12],
    "Short 5"
  ]
]

ADDITIONAL = [
    [
        [120, 40, 20, deg2rad(90), deg2rad(-5)],
        [300, 40, 15, deg2rad(-90), deg2rad(-5)],
        [],
        "Additional 1"
    ],
    [
        [120, 40, 20, deg2rad(90), deg2rad(-15)],
        [130, 120, 41, deg2rad(85), deg2rad(20)],
        [],
        "Additional 2"
    ]
]

In [23]:
%matplotlib qt
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.proj3d import proj_transform
from math import sin as sin
from math import cos as cos
import numpy as np
import pandas as pd
import sys, os
import time

# We import the file for visualizing the aircraft
# Including the following command to ensure that python is able to find the relevant files afer changing directory
sys.path.insert(0, '')
# Obtaining the current directory
cwd = os.getcwd()
# current_directory = Path(__file__).parent
# path_str = str(current_directory)

# Importing code for plotting
# rel_path = '..\Visualization'
rel_path = 'Visualization'
abs_path = os.path.abspath(os.path.join(cwd, rel_path))
os.chdir(abs_path)
from view_manager import ViewManager
from msg_state import MsgState
from visualization_simulation import generate_points_sphere, plot_trajectory
from rotations import euler_to_rotation

# Returning to initial directory
os.chdir(cwd)

# PROVIDING THE NAME OF THE INSTANCE TO BE CONSIDERED
data = 'Long 1'

# Providing the turning radius for pitch motion
Rpitch = 40

# Providing the length of the heading vector
length = 30.0 # 30.0 # Additional 2 - 30, Short 4 - 20

# Providing the scale of the aircraft model
scale = 10 # 15 # Additional 2 - 5, Short 4 - 15

plt.rcParams['text.usetex'] = True

class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs)
    
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)
    return arrow

setattr(Axes3D, 'arrow3D', _arrow3D)

# We obtain the initial and final configurations from the list of instances
if 'Long' in data:
    for inst in LONG:
        if inst[-1] == data:
            
            ini_config_arr = inst[0]
            fin_config_arr = inst[1]

elif 'Short' in data:
    for inst in SHORT:
        if inst[-1] == data:
            
            ini_config_arr = inst[0]
            fin_config_arr = inst[1]

elif 'Additional' in data:
    for inst in ADDITIONAL:
        if inst[-1] == data:
            
            ini_config_arr = inst[0]
            fin_config_arr = inst[1]

int_config_spacing = 45 # 120 for Short 4, 30 for Additional 2

for (ini_roll, fin_roll) in [(-15, 0), (0, 15), (15, -15)]:
# for (ini_roll, fin_roll) in [(15, -15)]:
# for (ini_roll, fin_roll) in [(15, -15)]:
    # for Ryaw in [30, 40, 50]:
    for Ryaw in [40]:

        # Loading the data
        df = pickle.load(open(data + ' ini_roll ' + str(ini_roll) + ' fin_roll ' + str(fin_roll) + ' Ryaw ' + str(Ryaw) + '.pickle', 'rb'))

        # We now plot the path
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        plt.tight_layout()

        # Make figure full screen
        manager = plt.get_current_fig_manager()
        manager.full_screen_toggle()

        # Obtaining the data
        path = df[1]
        tang_vect = df[2]
        tang_norm_vect = df[3]
        surf_norm_vect = df[4]
        path_type = df[-2]

        # We plot the initial location and heading and final location and heading
        ini_loc = np.array([ini_config_arr[0], ini_config_arr[1], ini_config_arr[2]])
        fin_loc = np.array([fin_config_arr[0], fin_config_arr[1], fin_config_arr[2]])
        ini_heading_angle = ini_config_arr[3]
        ini_pitch_angle = ini_config_arr[4]
        fin_heading_angle = fin_config_arr[3]
        fin_pitch_angle = fin_config_arr[4]

        # Computing the tangent vector
        ini_heading = np.array([cos(ini_heading_angle)*cos(ini_pitch_angle), sin(ini_heading_angle)*cos(ini_pitch_angle), sin(ini_pitch_angle)])
        fin_heading = np.array([cos(fin_heading_angle)*cos(fin_pitch_angle), sin(fin_heading_angle)*cos(fin_pitch_angle), sin(fin_pitch_angle)])

        # Computing the tangent normal vector
        ini_heading_norm = cos(ini_roll*math.pi/180)*np.array([cos(ini_heading_angle + math.pi/2), sin(ini_heading_angle + math.pi/2), 0])\
                + math.sin(ini_roll*math.pi/180)*np.cross(ini_heading, np.array([cos(ini_heading_angle + math.pi/2), sin(ini_heading_angle + math.pi/2), 0]))
        fin_heading_norm = cos(fin_roll*math.pi/180)*np.array([cos(fin_heading_angle + math.pi/2), sin(fin_heading_angle + math.pi/2), 0])\
                + math.sin(fin_roll*math.pi/180)*np.cross(fin_heading, np.array([cos(fin_heading_angle + math.pi/2), sin(fin_heading_angle + math.pi/2), 0]))

        # Computing the surface normal vector
        ini_norm = np.cross(ini_heading, ini_heading_norm)
        fin_norm = np.cross(fin_heading, fin_heading_norm)

        # Constructing the initial and final configuarations
        ini_config = np.array([ini_loc, ini_heading, ini_heading_norm, ini_norm])
        fin_config = np.array([fin_loc, fin_heading, fin_heading_norm, fin_norm])

        # Plotting the initial and final location and configuration
        ax.scatter(ini_loc[0], ini_loc[1], ini_loc[2], linewidth = 2, marker = 'o', s = 50, color = 'red', label = 'Initial location')
        ax.arrow3D(ini_loc[0], ini_loc[1], ini_loc[2], length*ini_heading[0],\
                        length*ini_heading[1], length*ini_heading[2], mutation_scale=20, fc='red', label = 'Tangent vector')
        ax.arrow3D(ini_loc[0], ini_loc[1], ini_loc[2], length*ini_heading_norm[0],\
                        length*ini_heading_norm[1], length*ini_heading_norm[2], mutation_scale=20, fc='blue', label = 'Tangent normal vector')
        ax.arrow3D(ini_loc[0], ini_loc[1], ini_loc[2], length*ini_norm[0],\
                        length*ini_norm[1], length*ini_norm[2], mutation_scale=20, fc='green', label = 'Surface normal vector')
        ax.scatter(fin_loc[0], fin_loc[1], fin_loc[2], linewidth = 2, marker = 'D', s = 50, color = 'blue', label = 'Final location')
        ax.arrow3D(fin_loc[0], fin_loc[1], fin_loc[2], length*fin_heading[0],\
                        length*fin_heading[1], length*fin_heading[2], mutation_scale=20, fc='red')
        ax.arrow3D(fin_loc[0], fin_loc[1], fin_loc[2], length*fin_heading_norm[0],\
                        length*fin_heading_norm[1], length*fin_heading_norm[2], mutation_scale=20, fc='blue')
        ax.arrow3D(fin_loc[0], fin_loc[1], fin_loc[2], length*fin_norm[0],\
                        length*fin_norm[1], length*fin_norm[2], mutation_scale=20, fc='green')

        # Plotting the path
        ax.plot3D([path[i][0] for i in range(len(path))], [path[i][1] for i in range(len(path))], [path[i][2] for i in range(len(path))], color = 'black', linewidth = 2, label = 'Path')

        # We also plot the configuration at every "int_config_spacing" instants
        for i in range(int_config_spacing, len(path), int_config_spacing):
            # ax.scatter(path[i][0], path[i][1], path[i][2], linewidth = 2, marker = 'o', s = 50, color = 'red')
            ax.arrow3D(path[i][0], path[i][1], path[i][2], length*tang_vect[i][0],\
                            length*tang_vect[i][1], length*tang_vect[i][2], mutation_scale=20, fc='red')
            ax.arrow3D(path[i][0], path[i][1], path[i][2], length*tang_norm_vect[i][0],\
                            length*tang_norm_vect[i][1], length*tang_norm_vect[i][2], mutation_scale=20, fc='blue')
            ax.arrow3D(path[i][0], path[i][1], path[i][2], length*surf_norm_vect[i][0],\
                            length*surf_norm_vect[i][1], length*surf_norm_vect[i][2], mutation_scale=20, fc='green')

        # We plot the aircraft
        true_state = MsgState()
        true_state.north = ini_loc[0]
        true_state.east = ini_loc[1]
        true_state.altitude = -ini_loc[2]
        true_state.theta = -ini_pitch_angle # NOTE: WE ARE DEFINING PITCH ANGLE IN THIS MANNER SINCE POSITIVE "PITCH ANGLE" IN
        # AEROSPACE DEPICTS ELEVATION ANGLE, WHICH IS OPPOSITE TO POSITIVE ROTATION ABOUT Y AXIS
        true_state.psi = ini_heading_angle
        true_state.phi = ini_roll*math.pi/180

        viewers = ViewManager(ax, animation=True, video=False, scale_aircraft=scale)
        viewers.update(fig, time.time(), true_state = true_state,  # true states
                )

        # ax.legend(fontsize = 14, loc = 0, ncol = 3)
        ax.legend(fontsize = 13, loc = 'upper center', ncol = 2)

        ax.tick_params(axis='both', which='major', labelsize = 14)
        ax.tick_params(axis='both', which='minor', labelsize = 14)

        ax.set_xlabel(r'$X$ (m)', fontsize = 18)
        ax.set_ylabel(r'$Y$ (m)', fontsize = 18)
        ax.set_zlabel(r'$Z$ (m)', fontsize = 18)

        # ax.view_init(elev = 13, azim = 31)
        # ax.view_init(elev = 5, azim = 81)
        # ax.view_init(elev = 35, azim = 30)

        # Viewing angles for Short 4
        # ax.view_init(elev = 39, azim = 64)

        # Trying for Additional 2
        # ax.view_init(elev = 28, azim = 114)
        # ax.view_init(elev = 40, azim = 123)
        # ax.view_init(elev = 33, azim = -100)
        
        # Final view 1 for Additional 2
        # ax.view_init(elev = 33, azim = -44)
        # Final view 2 for Additional 2 (second diagram in paper)
        # ax.view_init(elev = 30, azim = -75)

        # View for Long 1
        ax.view_init(elev = 44, azim = 0)

        # We plot the spheres around the vehicle
        # Considering spheres for the pitch motion
        if path_type in ['cyc_inner', 'plane_inner_outer', 'spheres_inner']:
            xfin, yfin, zfin = generate_points_sphere([ini_config[0, i] + Rpitch*ini_config[3, i] for i in range(3)], Rpitch)
            ax.plot_surface(xfin, yfin, zfin, color = 'orange', alpha=0.2)

        elif path_type in ['cyc_outer', 'plane_outer_inner', 'spheres_outer']:
            xfin, yfin, zfin = generate_points_sphere([ini_config[0, i] - Rpitch*ini_config[3, i] for i in range(3)], Rpitch)
            ax.plot_surface(xfin, yfin, zfin, color = 'magenta', alpha=0.2)

        # Considering spheres for the yaw motion
        elif path_type in ['cyc_left', 'plane_left_right', 'spheres_left']:
            xfin, yfin, zfin = generate_points_sphere([ini_config[0, i] + Ryaw*ini_config[2, i] for i in range(3)], Ryaw)
            ax.plot_surface(xfin, yfin, zfin, color = 'green', alpha=0.2)

        else:
            xfin, yfin, zfin = generate_points_sphere([ini_config[0, i] - Ryaw*ini_config[2, i] for i in range(3)], Ryaw)
            ax.plot_surface(xfin, yfin, zfin, color = 'blue', alpha=0.2)

        # Plotting the final spheres
        if path_type in ['cyc_inner', 'plane_outer_inner', 'spheres_inner']:
            xfin, yfin, zfin = generate_points_sphere([fin_config[0, i] + Rpitch*fin_config[3, i] for i in range(3)], Rpitch)
            ax.plot_surface(xfin, yfin, zfin, color = 'orange', alpha=0.2)
        elif path_type in ['cyc_outer', 'plane_inner_outer', 'spheres_outer']:
            xfin, yfin, zfin = generate_points_sphere([fin_config[0, i] - Rpitch*fin_config[3, i] for i in range(3)], Rpitch)
            ax.plot_surface(xfin, yfin, zfin, color = 'magenta', alpha=0.2)
        elif path_type in ['cyc_left', 'plane_right_left', 'spheres_left']:
            xfin, yfin, zfin = generate_points_sphere([fin_config[0, i] + Ryaw*fin_config[2, i] for i in range(3)], Ryaw)
            ax.plot_surface(xfin, yfin, zfin, color = 'green', alpha=0.2)
        else:
            xfin, yfin, zfin = generate_points_sphere([fin_config[0, i] - Ryaw*fin_config[2, i] for i in range(3)], Ryaw)
            ax.plot_surface(xfin, yfin, zfin, color = 'blue', alpha=0.2)

        plt.axis('equal')
        plt.tight_layout()

        plt.savefig(data + ' ini_roll ' + str(ini_roll) + ' fin_roll ' + str(fin_roll) + ' Ryaw ' + str(Ryaw) + '.png', dpi = 300)

        viewers.close()
        plt.show()

        # # We plot the pitch, yaw, and roll angles for the vehicle
        # fig = plt.figure()
        # pitch_angle_arr = np.zeros(len(path)); yaw_angle_arr = np.zeros(len(path)); roll_angle_arr = np.zeros(len(path))
        # for i in range(len(path)):

        #     # Constructing the rotation matrix
        #     R_mat = np.array([[tang_vect[i, 0], tang_norm_vect[i, 0], surf_norm_vect[i, 0]],\
        #                     [tang_vect[i, 1], tang_norm_vect[i, 1], surf_norm_vect[i, 1]],\
        #                     [tang_vect[i, 2], tang_norm_vect[i, 2], surf_norm_vect[i, 2]]])

        #     # We compute the orientation of the aircraft
        #     # Calculating the angles. The net rotation matrix is given below. Here, psi is yaw angle, theta is pitch, and phi is roll angle.
        #     """cψcθ −sψcφ + cψsθsφ sψsφ + cψsθcφ
        #     sψcθ cψcφ + sψsθsφ −cψsφ + sψsθcφ
        #     −sθ cθsφ cθcφ"""
        #     pitch_angle_val = np.nan; yaw_angle_val = np.nan; roll_angle_val = np.nan

        #     if math.sqrt((tang_vect[i][0])**2 + (tang_vect[i][1])**2) <= 10**(-8):
                
        #         pitch_angle = -np.sign(tang_vect[i][2])*math.pi/2
        #         # We set roll angle to be zero
        #         roll_angle = 0.0
        #         yaw_angle = math.atan2(-tang_norm_vect[i][0], tang_norm_vect[i][1])

        #         # We check if the desired rotation matrix matches
        #         R_net = euler_to_rotation(roll_angle, pitch_angle, yaw_angle)

        #         if abs(max(map(max, R_mat - R_net))) <= 10**(-4) and abs(min(map(min, R_mat - R_net))) <= 10**(-4):
                        
        #             pitch_angle_val = angle
        #             yaw_angle_val = yaw_angle
        #             roll_angle_val = roll_angle

        #     else:

        #         # pitch_angle = math.atan2(-tang_global_path[i][2], math.sqrt((tang_global_path[i][0])**2 + (tang_global_path[i][1])**2))
        #         # yaw_angle = math.atan2(tang_global_path[i][1], tang_global_path[i][0])
        #         # roll_angle = math.atan2(surf_normal_global_path[i][2], surf_normal_global_path[i][2])
        #         pitch_angle = [math.atan2(-tang_vect[i][2], math.sqrt((tang_vect[i][0])**2 + (tang_vect[i][1])**2)),\
        #                     math.atan2(-tang_vect[i][2], -math.sqrt((tang_vect[i][0])**2 + (tang_vect[i][1])**2))]
        #         for angle in pitch_angle:

        #             yaw_angle = math.atan2(tang_vect[i][1]/math.cos(angle), tang_vect[i][0]/math.cos(angle))
        #             roll_angle = math.atan2(tang_norm_vect[i][2]/math.cos(angle), surf_norm_vect[i][2]/math.cos(angle))

        #             # We check if the desired rotation matrix matches
        #             R_net = euler_to_rotation(roll_angle, angle, yaw_angle)

        #             if abs(max(map(max, R_mat - R_net))) <= 10**(-4) and abs(min(map(min, R_mat - R_net))) <= 10**(-4):
                        
        #                 pitch_angle_val = angle
        #                 yaw_angle_val = yaw_angle
        #                 roll_angle_val = roll_angle
        #                 break

        #     if np.isnan(pitch_angle_val):

        #         print('R is ', R_mat, ' and R_net is ', R_net)
        #         raise Exception("Could not find euler angles.")
            
        #     pitch_angle_arr[i] = -pitch_angle_val*180/math.pi
        #     yaw_angle_arr[i] = yaw_angle_val*180/math.pi
        #     roll_angle_arr[i] = roll_angle_val*180/math.pi

        # plt.plot(pitch_angle_arr, label = 'Pitch angle')
        # plt.plot(yaw_angle_arr, label = 'Yaw angle')
        # plt.plot(roll_angle_arr, label = 'Roll angle')
        # plt.xlabel('Time')
        # plt.ylabel('Angle [deg]')
        # plt.legend()
        # plt.show()


C:\Users\deepa\AppData\Local\Temp\ipykernel_36780\1394121919.py:266: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [8]:
rel_path = 'Visualization'
abs_path = os.path.abspath(os.path.join(cwd, rel_path))
os.chdir(abs_path)
from view_manager import ViewManager
from msg_state import MsgState
from visualization_simulation import generate_points_sphere, plot_trajectory

# Returning to initial directory
os.chdir(cwd)

# Saving the video
plot_trajectory(ini_config, fin_config, path, tang_vect, tang_norm_vect, surf_norm_vect, path_type, Ryaw, Rpitch,\
                xgrid_size = [300, 700], ygrid_size = [-400, 0], zgrid_size = [250, 650], length_vec_orientation = 30,\
                scale_aircraft = 15, elev = 25, azim = 2, video_name = False) # 'Short_4_motion_constraint_Ryaw_30.mp4')

KeyboardInterrupt: 